In [1]:
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, Layout
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
client = Client("GEONET")

In [4]:
current = False

In [5]:
#plot initialisation
filt = False
corners = 2
zerophase = False
defscale = 5e3
scale = defscale
size = (2400,1200)

In [6]:
def selsite(s):
  global site, stn, loc, cmp, net
  site = s['new']
  
  stn = site.split(".")[0]
  loc = site.split(".")[1]
  cmp = site.split(".")[2]
  net = site.split(".")[3]

In [7]:
def seldate(d):
  global pdate
  date = d['new']
  pdate = date.strftime("%Y%m%d")
  

In [8]:
def retrieve(r):
  global st
  if (current):   #current data
    time = UTCDateTime()
    st = client.get_waveforms(str(net), str(stn), str(loc), str(cmp), time - 86400, time)
  else:   #older data
    time = UTCDateTime(pdate)
    st = client.get_waveforms(str(net), str(stn), str(loc), str(cmp), time, time + 86400)
  print 'data retrieved'
  print str(st)

In [9]:
def bandpass(bp):
  global lp, hp
  lp, hp  = bp['new']

In [10]:
def filter(f):
  global filt  
  filt = f['new']

In [11]:
def gain(g):
  global magn, scale  
  magn = g['new']
  scale = defscale / magn

In [12]:
def screen(c):
    if (filt):
        stcp = st.copy()
        stcp.filter('bandpass', freqmin=lp, freqmax=hp, corners=corners, zerophase=zerophase)
        title = site+'_'+pdate+'_bandpass_'+str(lp)+'-'+str(hp)
        stcp.plot(type='dayplot', vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title)
    else:
        title = site + '_' + pdate + '_nofilt'
        st.plot(type='dayplot', vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title)

In [13]:
def output(o):
    if (filt):
        stcp = st.copy()
        stcp.filter('bandpass', freqmin=lp, freqmax=hp, corners=corners, zerophase=zerophase)
        title = site+'_'+pdate+'_bandpass_'+str(lp)+'-'+str(hp)
        stcp.plot(type='dayplot', vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title, outfile=title+'.drum.png')
    else:
        title = site+'_'+pdate+'_nofilt'
        st.plot(type='dayplot', vertical_scaling_range= scale, size=size, color=('navy','blue','blue','blue'), title=title, outfile=title+'.drum.png')

In [14]:
s = widgets.Select(
    options=['select stream', 'WIZ.10.HHZ.NZ', 'WSRZ.10.HHZ.NZ', 'MAVZ.10.HHZ.NZ', 'FWVZ.10.HHZ.NZ', 'WHVZ.10.HHZ.NZ', 'TRVZ.10.HHZ.NZ', 'GLKZ.10.HHZ.NZ'],
    value='select stream',
    description='Stream to display:',
    disabled=False
)
s.observe(selsite, names='value')

d = widgets.DatePicker(
  description='Date to display:')
d.observe(seldate, names='value')

r=widgets.Button(
    description='Retrieve data',
    disabled=False,
    button_style='success',
    tooltip='Retrieve data for plotting')
r.on_click(retrieve)

widgets.HBox([d, s, r])

data retrieved
2 Trace(s) in Stream:
NZ.WIZ.10.HHZ | 2017-05-23T23:59:58.863241Z - 2017-05-24T19:28:01.603241Z | 100.0 Hz, 7008275 samples
NZ.WIZ.10.HHZ | 2017-05-24T19:27:57.473127Z - 2017-05-24T20:28:02.703127Z | 100.0 Hz, 360524 samples


In [15]:
f=widgets.Checkbox(
    description='Filter?',
    value=False,
    tooltip='Set filtering',
    disabled=False,
    layout=widgets.Layout(width='10%'))
f.observe(filter,names='value')

b=widgets.FloatRangeSlider(
    value=[0.1, 10],
    min=0.05,
    max=10,
    step=0.05,
    description='band-pass:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    slider_color='white')
b.observe(bandpass, names='value')

g=widgets.FloatSlider(
    value=1,
    min=0.1,
    max=10,
    step=0.1,
    description='plot gain:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    slider_color='white')
g.observe(gain, names='value')

widgets.HBox([f,b,g])

c=widgets.Button(
    description='Show on screen',
    disabled=False,
    button_style='success',
    tooltip='Make drumplot')
c.on_click(screen)

o=widgets.Button(
    description='Output to file',
    disabled=False,
    button_style='success',
    tooltip='Make file copy')
o.on_click(output)

widgets.HBox([f, b, g, c, o])